# Importing Libraries

In [ ]:
import numpy as np # Linear algebra
import pandas as pd # Tabular data
import matplotlib.pyplot as plt # Data visualization
import seaborn as sns # High-level data visualization
import PIL # Image manipulation
import time # Timing
from tensorflow.keras.models import load_model # Loading model
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Object containing data
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report # Model metrics 

# Loading Model Weights and Testing Data

In [ ]:
# Using tensorflow to load pretrained model weights
baseline_CNN_model = load_model("/kaggle/input/grapevine-disease-detection-model-weights/model_weights/baseline_CNN_weights.hdf5")
DenseNet121_model = load_model("/kaggle/input/grapevine-disease-detection-model-weights/model_weights/DenseNet121_weights.hdf5")
EfficientNetB7_model = load_model("/kaggle/input/grapevine-disease-detection-model-weights/model_weights/EfficientNetB7_weights.hdf5")
MobileNetV2_model = load_model("/kaggle/input/grapevine-disease-detection-model-weights/model_weights/MobileNetV2_weights.hdf5")
ResNet50_model = load_model("/kaggle/input/grapevine-disease-detection-model-weights/model_weights/ResNet50_weights.hdf5")
VGG16_model = load_model("/kaggle/input/grapevine-disease-detection-model-weights/model_weights/VGG16_weights.hdf5")

In [ ]:
# List containing disease names
list_of_classes = ['Black Rot', 'ESCA', 'Leaf Blight', 'Healthy']

In [ ]:
# Datagen that will be used for model predictions and efficiency test
test_datagen = ImageDataGenerator().flow_from_directory('/kaggle/input/grape-disease-dataset-original/Original Data/test',
                                                        target_size = (224,224),
                                                        batch_size = 32,
                                                        class_mode = 'sparse',
                                                        classes = list_of_classes,
                                                        shuffle = False)

In [ ]:
# Used to make test images (batch_size is equal to total number of test images)
plotting_datagen = ImageDataGenerator().flow_from_directory('/kaggle/input/grape-disease-dataset-original/Original Data/test',
                                                            target_size = (224,224),
                                                            batch_size = 1805,
                                                            class_mode = 'sparse',
                                                            classes = list_of_classes,
                                                            shuffle = False)

In [ ]:
# Object containing all testing images that is used for error visualization
test_images = plotting_datagen[0][0]

# Model Evaluation Pipeline

In [ ]:
def evaluate_model(model, model_name, model_predictions=None, error_rows=4, error_cols=4):
    ### Accuracy score:
    try:
        # Checking if model_predictions is None
        model_predictions[0]
    except:
        # Getting model_predictions from the model and test_datagen
        model_predictions = model.predict(test_datagen, verbose=0) 
        # Converting probabilistic predictions to integers
        final_model_predictions = np.argmax(model_predictions, axis=-1)
    else:
        # Execute when model_predictions has been given
        final_model_predictions = model_predictions 
    # Calculating model_accuracy using accuracy_score function from sklearn.metrics
    model_accuracy = round(accuracy_score(final_model_predictions, test_datagen.labels)*100, 3)
    # Printing model_name and model_accuracy
    print(f'The {model_name} had an accuracy of {model_accuracy}%.', end='\n\n')
    
    ### Confusion matrix
    # Getting model_confusion_matrix using confusion_matrix function from sklearn.metrics
    model_confusion_matrix = confusion_matrix(y_true=test_datagen.labels,y_pred=final_model_predictions)
    # Setting figsize and dpi for the plot
    plt.figure(figsize=(2,2), dpi=96)
    # Adding the title
    plt.title(f'{model_name}')
    # Rotating xticks
    plt.xticks(rotation=0)
    # Plotting heatmap
    sns.heatmap(model_confusion_matrix, vmin=0, vmax=500, annot=True, fmt='d',
                xticklabels=list_of_classes, yticklabels=list_of_classes, cbar=False)
    # Adding x and y labels
    plt.xlabel('Predicted Disease')
    plt.ylabel('Actual Disease')
    # Displaying plot
    plt.show()
    
    ### Classification report
    print(classification_report(y_true=test_datagen.labels,y_pred=final_model_predictions))
    
    ### Error visualization
    # Getting indices of model errors
    model_errors = (final_model_predictions - test_datagen.labels != 0)
    # Getting the corresponding images
    model_image_errors = test_images[model_errors]
    # Getting the predicted labels
    model_pred_errors = np.array(final_model_predictions)[model_errors]
    # Getting the true labels
    model_actual_errors = test_datagen.labels[model_errors]
    # Function that adds a frame around the image
    def frame_image(image, frame_width):
        # Getting image dimensions in pixels
        y_pixels, x_pixels = image.shape[0], image.shape[1]
        # Creating an black RGB image with dimensions frame_width*2 + y_pixels and frame_width*2 + x_pixels
        empty_image = PIL.Image.new('RGB', (frame_width*2+y_pixels,frame_width*2+x_pixels), (0,0,0))
        # Converting the empty image to a numpy array
        framed_image = np.array(empty_image.getdata()).reshape(empty_image.size[0], empty_image.size[1], 3)
        # Adding the original image inside the frame
        framed_image[frame_width:-frame_width, frame_width:-frame_width] = image
        # Returning the framed image
        return framed_image
    # Error index
    n = 0
    # Subplots
    fig, ax = plt.subplots(error_rows,error_cols,figsize=(7.5,2.25*error_rows), dpi=96)
    # Iterating over rows
    for row in range(error_rows):
        # Iterating over columns
        for col in range(error_cols):
            try:
                # Check if there are multiple error_rows
                if error_rows > 1:
                    # Add the title
                    ax[row,col].set_title(f'Predicted label: {model_pred_errors[n]}\nTrue label: {model_actual_errors[n]}')
                    # Plot the image
                    ax[row,col].imshow(frame_image(model_image_errors[n],5))
                    # Set the aspect ratio
                    ax[row,col].set_aspect(aspect=1)
                else:
                    # Add the title
                    ax[col].set_title(f'Predicted label: {model_pred_errors[n]}\nTrue label: {model_actual_errors[n]}')
                    # Plot the image
                    ax[col].imshow(frame_image(model_image_errors[n],5))
                    # Set the aspect ratio
                    ax[col].set_aspect(aspect=1)
                # Increment the error index
                n += 1
            except:
                # Do nothing
                pass
            # Check if there are multiple error_rows
            if error_rows > 1:
                # Remove axis
                ax[row,col].axis('off')
            else:
                # Remove axis
                ax[col].axis('off')
    # Display the subplots
    plt.show()
    # Return the model predictions and model accuracy
    return final_model_predictions, model_accuracy

# Model Evaluation

In [ ]:
# Plot parameters
plt.rc('xtick', labelsize=6)
plt.rc('ytick', labelsize=6)

In [ ]:
# Calling the evaluate_model function
baseline_CNN_preds, baseline_CNN_acc = evaluate_model(model = baseline_CNN_model,
                                                      model_name = 'Baseline CNN',
                                                      error_rows = 4)

In [ ]:
# Calling the evaluate_model function
DenseNet121_preds, DenseNet121_acc = evaluate_model(model = DenseNet121_model,
                                                    model_name = 'DenseNet121',
                                                    error_rows = 4)

In [ ]:
# Calling the evaluate_model function
EfficientNetB7_preds, EfficientNetB7_acc = evaluate_model(model = EfficientNetB7_model,
                                                          model_name = 'EfficientNetB7',
                                                          error_rows = 2)

In [ ]:
# Calling the evaluate_model function
MobileNetV2_preds, MobileNetV2_acc = evaluate_model(model = MobileNetV2_model,
                                                    model_name = 'MobileNetV2',
                                                    error_rows = 4)

In [ ]:
# Calling the evaluate_model function
ResNet50_preds, ResNet50_acc = evaluate_model(model = ResNet50_model,
                                              model_name = 'ResNet50',
                                              error_rows = 4)

In [ ]:
# Calling the evaluate_model function
VGG16_preds, VGG16_acc = evaluate_model(model = VGG16_model,
                                        model_name = 'VGG16',
                                        error_rows = 4)

# Comparing Models

In [ ]:
# Dictionary containing accuracies of the models
accuracies = {'Model':['Baseline CNN', 'DenseNet121', 'EfficientNetB7', 'MobileNetV2', 'ResNet50', 'VGG16'],
              'Accuracy':[baseline_CNN_acc, DenseNet121_acc, EfficientNetB7_acc, MobileNetV2_acc, ResNet50_acc, VGG16_acc]}
# Converting dictionary to a dataframe
accuracies = pd.DataFrame(accuracies)
# Sorting by accuracy
accuracies = accuracies.sort_values('Accuracy',ascending=False)

In [ ]:
# Plot parameters
plt.rc('xtick', labelsize=8)
plt.rc('ytick', labelsize=8)

In [ ]:
# Setting figsize and dpi for the plot
plt.figure(figsize=(7,5), dpi=100)
# Adding the title
plt.title('Accuracies of Different Models')
# Rotating xticks
plt.xticks(rotation=0)
# Setting y limits
plt.ylim([90,100])
# Plotting barplot
sns.barplot(data=accuracies, x='Model', y='Accuracy', palette='inferno')
# Displaying plot
plt.show()

In [ ]:
# Dictionary containing model sizes of the models
model_sizes = {'Model Size (MB)':[28.91, 257.8, 9.4, 94.78, 58.94],
               'Accuracy':[DenseNet121_acc, EfficientNetB7_acc, MobileNetV2_acc, ResNet50_acc, VGG16_acc]}
# Converting dictionary to a dataframe
model_sizes = pd.DataFrame(model_sizes)
# Sorting by model sizes
model_sizes = model_sizes.sort_values('Model Size (MB)',ascending=False)

In [ ]:
# Reset plot parameters
plt.rcdefaults()

In [ ]:
# Setting figsize and dpi for the plot
plt.figure(figsize=(7,5), dpi=100)
# Adding the title
plt.title('Model Size vs Accuracy')
# Rotating xticks
plt.xticks(rotation=0)
# Setting y limits
plt.ylim([90,100])
# Plotting regplot
sns.regplot(data=model_sizes, x='Model Size (MB)', y='Accuracy', color='b', lowess=True)
# Displaying plot
plt.show()

# Max-Voting Ensemble

In [ ]:
def max_voting_ensemble(pred1,pred2,pred3):
    # Will contain final predictions
    final_pred = []
    # Iterating over length of testing data
    for i in range(1805):
        # Will contain prediction counts
        pred_counts = []
        # Iterating over each disease
        for disease in range(4):
            # Appending count of the disease
            pred_counts.append([pred1[i], pred2[i], pred3[i]].count(disease))
        # Appending argmax of prediction counts
        final_pred.append(np.array(pred_counts).argmax())
    # Returning final predictions
    return final_pred

In [ ]:
# Calling max_voting_ensemble function
max_voting_preds = max_voting_ensemble(EfficientNetB7_preds, baseline_CNN_preds, ResNet50_preds)

In [ ]:
# Plot parameters
plt.rc('xtick', labelsize=8)
plt.rc('ytick', labelsize=8)

In [ ]:
# Calling the evaluate_model function
max_voting_preds, max_voting_accuracy_score = evaluate_model(model = None,
                                                             model_name = 'Max-Voting Ensemble',
                                                             model_predictions = max_voting_preds,
                                                             error_rows = 1)

# Efficiency Test

In [ ]:
# Getting current time
start = time.time()
# Getting EfficientNetB7 preds
EfficientNetB7_model_preds = np.argmax(EfficientNetB7_model.predict(test_datagen, verbose=0), axis=-1)
# Getting ResNet50 preds
ResNet50_model_preds = np.argmax(ResNet50_model.predict(test_datagen, verbose=0), axis=-1)
# Getting baseline CNN preds
baseline_CNN_model_preds = np.argmax(baseline_CNN_model.predict(test_datagen, verbose=0), axis=-1)
# Calling max_voting_ensemble function
final_predictions = max_voting_ensemble(EfficientNetB7_model_preds,ResNet50_model_preds,baseline_CNN_model_preds)
# Getting current time
end = time.time()
# Calculating average time to predict a single image
print(f'It takes the max-voting ensemble {round((end-start)/1.805,2)} milliseconds to predict a single image.')